In [401]:
library(readxl)
library(data.table)
library(ggplot2)
library(ggprism)
library(ggforce)
library(ggetho)
library(ggsignif)
library(zoo)
theme_set(theme_prism(base_size=20))
options(repr.plot.width=15, repr.plot.height=15)

In [402]:
idoc_dir <- "/idoc_data/results/IDOC_001/"
xlsx_file <- "IDOC_METHODS_PAPER_METADATA.xlsx"

In [403]:
sheets <- c("20min STM", "1hr STM", "2hr STM", "3hr STM")
zts <- paste0("ZT", as.character(round(seq(4, 30, .5), 1)))

dts <- lapply(sheets, function(sheet) {
    x <- cbind(
            as.data.table(suppressMessages(readxl::read_xlsx(xlsx_file, sheet = sheet, skip = 2)))[, .(Fly_group_experiment_treatment, fly_name_reference, Genotype = "Iso31", PRE, POST)],
            experiment = sheet,
            `Training-session` = "1X"
        )
    for (zt in zts) {
        x[[zt]] <- NA_real_
    } 
    for (zt in zts) {
        x[[paste0(zt, "_interactions")]] <- NA_real_
    }

    x
    

})
names(dts) <- sheets

In [404]:
# idoc_folders <- list.files(idoc_dir, full.names = T)
# # idoc_folders <- idoc_folders[grep(pattern = "2021", x = idoc_folders)]
# # print(idoc_folders)
# ltm_chx_folders <- c("2021-11-27 (CH1)", "2021-11-28 (CH)", "2021-11-30 (CH)", "2021-12-02 (CH)", "2021-11-17 (CH)", "2021-12-22 (CH)", "2021-11-28 (CH)")
# ltm_chx_folders <- names(sapply(ltm_chx_folders, function(folder) {idoc_folders[grep(x = idoc_folders, pattern = folder)]}))

# ltm_chx_folders <- sapply(ltm_chx_folders, function(folder) {file.path(idoc_dir,folder)})



# ltm_chx <-lapply(
#     ltm_chx_folders,
#     function(idoc_folder) {
#         exp_date <- substr(basename(idoc_folder), 1, 10)
#         subexperiments <- sort(list.files(idoc_folder, full.names = T))

#         # print(idoc_folder)
#         # print(exp_date)
#         # print(subexperiments)
#         if ((substr(basename(subexperiments[7]), 1, 10) != exp_date) | (substr(basename(subexperiments[8]), 1, 10) == exp_date)) {
#             print(subexperiments[7])
#             print(exp_date)
#             stop(paste0(idoc_folder, " is corruped"))
#         }

#         paths <- list.files(subexperiments[1], full.names = T)
#         csv_file <- paths[grep(pattern = "_PI.csv", x = paths)]
#         data1 <- data.table::fread(csv_file)
#         data1 <- cbind(exp_folder=basename(idoc_folder), data1)
#         colnames(data1)[colnames(data1) == "preference_index"] <- "PRE"
#         data1[, apetitive:=NULL]
#         data1[, aversive:=NULL]
        
        
        
#         paths <- list.files(subexperiments[8], full.names = T)
#         csv_file <- paths[grep(pattern = "_PI.csv", x = paths)]
#         data2 <- data.table::fread(csv_file)
#         data2 <- cbind(exp_folder=basename(idoc_folder), data2)
#         colnames(data2)[colnames(data2) == "preference_index"] <- "POST"
#         data2[, apetitive:=NULL]
#         data2[, aversive:=NULL]
        
#         data <- merge(data1, data2, by=c("exp_folder", "region_id"))
        
#         data[, Fly_group_experiment_treatment := "ISO31\'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Spaced_ETHOSCOPE-NO_CHX"]
#         data[, fly_name_reference := paste0(basename(idoc_folder), "_ROI_", region_id)]
#         data[, region_id := NULL]
#         data[, exp_folder := NULL]
#         data[, experiment := "LTM_CHX"]
#         data[, `Training-session` := "6X_Spaced"]
#         for (zt in zts) {
#             data[[zt]] <- NA_real_
#         } 
#         data
        
# })
# ltm_chx <- do.call(rbind, ltm_chx)
# ltm_chx <- ltm_chx[, c("Fly_group_experiment_treatment", "fly_name_reference", "PRE", "POST", "experiment", "Training-session", zts), with=F]

In [405]:
ltm <- as.data.table(suppressMessages(readxl::read_xlsx(xlsx_file, sheet = "LTM experiments", skip = 2)))
ltm$experiment <- "LTM"

In [406]:
ltm <- ltm[!is.na(folder_name),]


In [407]:
ltm[is.na(`Training-session`), `Training-session` := "NONE"]

In [408]:

ltm <- ltm[!is.na(`Training-session`),]
ltm <- ltm[SD_quality %in% c("No_sleep_depriver", "good") & SD_quality_second_eye %in% c("No_sleep_depriver", "good"),]
nrow(ltm)
ltm<-ltm[ substr(fly_name_reference, 1, 10) !="2023-09-12",]
ltm<-ltm[ substr(fly_name_reference, 1, 10) !="2023-09-13",]
ltm$Genotype <- "Iso31"
ltm <- ltm[, c("Fly_group_experiment_treatment", "fly_name_reference", "Genotype", "PRE", "POST", "experiment","Training-session", zts), with=F]
for (col in paste0(zts, "_interactions")) {
    ltm[[col]] <- NA
}

[1] 319

In [409]:
gtacr <- as.data.table(suppressMessages(readxl::read_xlsx(xlsx_file, sheet = "LTM_GTACRS2_Sleep", skip = 2)))

In [410]:
sum(colnames(gtacr) == "POST")

[1] 1

In [411]:
gtacr <- gtacr[, c("Fly_group_experiment_treatment", "fly_name_reference", "Fly_genotype", "PRE", "POST", "Test", "comment on SD", "interactor_time_window", "Training-session", zts, paste0(zts, "_interactions")), with=F]

In [412]:
gtacr[, Genotype := Fly_genotype]
gtacr[, Fly_genotype := NULL]

In [413]:
gtacr[substr(Genotype, 1, 5) == "ISO31", Genotype := "Iso31"]

In [414]:
gtacr[, color := NA_character_]
gtacr[, voltage := NA_real_]
gtacr[, stimulator := NA_character_]

In [415]:
gtacr[grep(pattern = "Blue", x = gtacr$`comment on SD`), color := "Blue"]

In [416]:
gtacr[grep(pattern = "3.5V", x = gtacr$`comment on SD`), voltage := 3.5]
gtacr[grep(pattern = "4V", x = gtacr$`comment on SD`), voltage := 4]
gtacr[grep(pattern = "4.5V", x = gtacr$`comment on SD`), voltage := 4.5]

In [417]:
gtacr[grep(pattern = "SleepStimulator", x = gtacr$Fly_group_experiment_treatment), stimulator := "SleepStimulator"]
gtacr[grep(pattern = "AwakeStimulator", x = gtacr$Fly_group_experiment_treatment), stimulator := "AwakeStimulator"]

In [418]:
gtacr[, experiment := paste0("LTM GTACR ", color, " ", voltage, " ", stimulator)]

In [419]:
gtacr <- gtacr[, c("Fly_group_experiment_treatment", "fly_name_reference", "Genotype", "PRE", "POST", "experiment", "Training-session", zts, paste0(zts, "_interactions")), with=F]

In [420]:
colnames(gtacr) == colnames(ltm)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [421]:
ltm <- rbind(ltm, gtacr)

In [422]:
table(ltm$`Training-session`)


  6X_Massed   6X_Spaced No_training        NONE 
         50         266          10          31 

In [423]:
dts$LTM <- ltm

In [424]:
data <- do.call(rbind, dts)

In [425]:
# data[, table(Fly_group_experiment_treatment)]

In [478]:
data_clean <- data[!is.na(PRE) & !is.na(POST) & PRE != "NA" & POST != "NA" & PRE != "#VALUE!" & POST != "#VALUE!",]

In [479]:
data_clean$PRE <- as.numeric(data_clean$PRE)
data_clean$POST <- as.numeric(data_clean$POST)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [480]:
data_clean[, table(abs(PRE) < 0.2)]


FALSE  TRUE 
   33   304 

In [481]:
data_clean[, table(`Training-session`)]

Training-session
       1X 6X_Massed 6X_Spaced      NONE 
      169        29       140        31 

In [482]:
# data_clean[`Training-session`=="6X_Massed"]

In [483]:
data_clean <- data_clean[(abs(PRE) < 0.2) | `Training-session` == "NONE" ,]

In [484]:
data_clean[, table(`Training-session`)]

Training-session
       1X 6X_Massed 6X_Spaced      NONE 
      146        26       132        31 

In [485]:
data_clean$id <- 1:nrow(data_clean)

In [486]:
data_clean[fly_name_reference %in% data_clean[duplicated(fly_name_reference), unique(fly_name_reference)], fly_name_reference := paste0(fly_name_reference, "_", 1:.N), by=fly_name_reference]

In [487]:
data_clean[`Training-session` == "NONE", PRE := 0]
data_clean[`Training-session` == "NONE", POST := 0]

In [488]:
sum(duplicated(data_clean$fly_name_reference))

[1] 0

In [489]:
sum(is.na(data_clean$fly_name_reference))

[1] 0

In [490]:
columns <- c("Fly_group_experiment_treatment", "fly_name_reference", "experiment", "id", "PRE", "POST", "Training-session", "Genotype", zts, paste0(zts, "_interactions"))

In [491]:
data_clean_long <- melt(data = data_clean[, columns, with=F], measure.vars = c("PRE", "POST"), value.name = "PI", variable.name = "test")

In [492]:
data_clean_long<-data_clean_long[!is.na(PI),]


In [493]:
data_clean_long$test <- factor(as.character(data_clean_long$test), c("PRE", "POST"))

In [494]:
gtacr_experiments <- unique(data_clean_long$experiment)[substr(unique(data_clean_long$experiment), 1, 9) == "LTM GTACR"]
data_clean_long$experiment <- factor(data_clean_long$experiment, levels=c(sheets, "LTM", "LTM CHX", gtacr_experiments))

In [495]:
table(data_clean_long$experiment)


                         20min STM                            1hr STM 
                                98                                 60 
                           2hr STM                            3hr STM 
                                24                                110 
                               LTM                            LTM CHX 
                               284                                  0 
LTM GTACR Blue 4.5 SleepStimulator   LTM GTACR Blue 4 SleepStimulator 
                                36                                 34 
LTM GTACR Blue 3.5 SleepStimulator 
                                24 

In [496]:
learning <- sapply(1:nrow(data_clean_long), function(x) {"NONE"})
learning[grep(pattern = "1X", x = data_clean_long$Fly_group_experiment_treatment)] <- "1X"
learning[grep(pattern = "6X_Spaced", x = data_clean_long$Fly_group_experiment_treatment)] <- "6X_Spaced"
data_clean_long$learning <- learning

In [497]:
interval <- sapply(1:nrow(data_clean_long), function(x) {"NONE"})
interval[grep(pattern = "ZT05-ZT24", x = data_clean_long$Fly_group_experiment_treatment)] <- "ZT05-ZT24"
interval[grep(pattern = "ZT05-ZT11", x = data_clean_long$Fly_group_experiment_treatment)] <- "ZT05-ZT11"
interval[grep(pattern = "ZT12-ZT18", x = data_clean_long$Fly_group_experiment_treatment)] <- "ZT12-ZT18"

data_clean_long$interval <- factor(interval, levels=c("ZT05-ZT11", "ZT12-ZT18", "ZT05-ZT24", "NONE"))

In [498]:
data_clean_long[, Training_session :=  `Training-session`]

In [499]:
table(data_clean_long$Training_session)


       1X 6X_Massed 6X_Spaced      NONE 
      292        52       264        62 

In [500]:
data_clean_long[, `Training-session` := NULL]

In [501]:
data_clean_long <- data_clean_long[, c("Fly_group_experiment_treatment", "fly_name_reference",	"experiment", "id", "test", "PI", "Training_session", "Genotype", "interval", zts, paste0(zts, "_interactions")), with=F]

In [502]:
table(data_clean_long$Training_session)


       1X 6X_Massed 6X_Spaced      NONE 
      292        52       264        62 

In [503]:
chui_data <- data.table::fread(file = "chui_tidy_data.csv")

In [505]:
table(chui_data$experiment)


         20min STM 20min STM unpaired                LTM            LTM CHX 
               238                108                164                126 

In [506]:
chui_data$User <- "CH"
data_clean_long[grep(x=fly_name_reference, pattern="ESB"), User := "ESB"]
data_clean_long[grep(x=fly_name_reference, pattern="ET"), User := "ET"]
data_clean_long[grep(x=fly_name_reference, pattern="AOJ"), User := "AOJ"]

In [507]:
all_data <- rbind(data_clean_long, chui_data)

In [508]:
table(all_data$Training_session)


       1X 6X_Massed 6X_Spaced      NONE 
      638        52       554        62 

In [509]:
all_data[, table(User)]

User
AOJ  CH ESB  ET 
 38 636 180 390 

In [512]:
all_data[Training_session=="6X_Massed",]

Fly_group_experiment_treatment,fly_name_reference,experiment,id,test,PI,Training_session,Genotype,interval,ZT4,⋯,ZT26_interactions,ZT26.5_interactions,ZT27_interactions,ZT27.5_interactions,ZT28_interactions,ZT28.5_interactions,ZT29_interactions,ZT29.5_interactions,ZT30_interactions,User
<chr>,<chr>,<fct>,<int>,<fct>,<dbl>,<chr>,<chr>,<fct>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_2,LTM,187,PRE,0.09523810,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_3,LTM,188,PRE,-0.01587302,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_4,LTM,189,PRE,0.10000000,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_5,LTM,190,PRE,0.10000000,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_7,LTM,191,PRE,-0.06666667,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_8,LTM,192,PRE,0.00000000,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_9,LTM,193,PRE,-0.10000000,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_10,LTM,194,PRE,0.05555556,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB
ISO31'_LTM(24hr)_OCT_CSplus_EShock_US_Paired_6X_Massed_DefaultStimulator _No_stimulatorETHOSCOPE-YES,2023-07-23_16-20-43(ESB-massed)_ROI_11,LTM,195,PRE,-0.12500000,6X_Massed,Iso31,NONE,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,ESB


In [513]:
all_data<-all_data[!is.na(PI),]

In [514]:
table(all_data$Training_session)


       1X 6X_Massed 6X_Spaced      NONE 
      638        52       554        62 

In [515]:
counts <- all_data[, .N, by=fly_name_reference]

In [516]:
# all_data[fly_name_reference %in% counts[N!=2, fly_name_reference],]
         

In [517]:
all_data <- all_data[fly_name_reference %in% setdiff(fly_name_reference, counts[N!=2, fly_name_reference]),]

In [518]:
counts <- all_data[, .N, by=fly_name_reference]
counts[, table(N)]

N
  2 
629 

In [519]:
table(all_data$Training_session)


       1X 6X_Massed 6X_Spaced      NONE 
      608        52       536        62 

In [520]:
data.table::fwrite(x = all_data, file = "tidy_data.csv")

In [521]:
all_data[Genotype=="orb2", table(experiment)]

experiment
                         20min STM                            1hr STM 
                                22                                  0 
                           2hr STM                            3hr STM 
                                 0                                  0 
                               LTM                            LTM CHX 
                                34                                  0 
LTM GTACR Blue 4.5 SleepStimulator   LTM GTACR Blue 4 SleepStimulator 
                                 0                                  0 
LTM GTACR Blue 3.5 SleepStimulator                 20min STM unpaired 
                                 0                                  0 

In [222]:
table(all_data$Genotype)


                        Iso31 MB010B.(II)SPARC-Chrimson ISO 
                          200                           328 
     MB010B.(II)SPARC-GFP ISO                          orb2 
                           66                            56 

In [223]:
all_data[is.na(PI),]

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


Fly_group_experiment_treatment,fly_name_reference,experiment,id,test,PI,Training_session,Genotype,interval,ZT4,⋯,ZT26_interactions,ZT26.5_interactions,ZT27_interactions,ZT27.5_interactions,ZT28_interactions,ZT28.5_interactions,ZT29_interactions,ZT29.5_interactions,ZT30_interactions,User
<chr>,<chr>,<fct>,<int>,<fct>,<dbl>,<chr>,<chr>,<fct>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


In [522]:
table(all_data$experiment)


                         20min STM                            1hr STM 
                               306                                 60 
                           2hr STM                            3hr STM 
                                24                                110 
                               LTM                            LTM CHX 
                               430                                126 
LTM GTACR Blue 4.5 SleepStimulator   LTM GTACR Blue 4 SleepStimulator 
                                36                                 34 
LTM GTACR Blue 3.5 SleepStimulator                 20min STM unpaired 
                                24                                108 